In [4]:
import sys, os, pprint, glob, re

In [5]:
IN_PATH = '/home/seki/_seki/DSO/in.txt'

f = open(IN_PATH, 'r')
buf = f.readlines()
f.close()
pprint.pprint(buf)

['AAA O\n',
 'BBBBB O\n',
 'CC B-GOODSNAME 1_2\n',
 'DDD I-GOODSNAME 1_2\n',
 'EEE I-GOODSNAME 1_2\n',
 'FFFFF E-GOODSNAME 1_2\n',
 'G O\n',
 'H O\n',
 'I O\n',
 'J O\n',
 'KKK O\n',
 'LL O\n',
 'MMMM S-ORDERCURRENCY 1\n',
 'NNNNN S-ORDERPRICE 1\n',
 'OOOO B-ORDERUNIT 1\n',
 'PPPPPP E-ORDERUNIT 1\n',
 'QQQQQQQQQQQQ O\n',
 'RR O\n',
 'SS O\n',
 'TT O\n',
 'UU O\n',
 'V S-ORDERCURRENCY 2\n',
 'W S-ORDERUNIT 2\n',
 'X O\n',
 'YYY B-ORDERUNIT 3\n',
 'ZZZZZZZZZZZZZZZZ E-ORDERUNIT 3']


In [3]:
outs = []
pre_word = ''
pre_label = ''

word_sep = '_'
item_sep = ' ' 

for i, data in enumerate(buf):
    
    lists = re.sub(r'\n$', '', data).split(' ')
    
    if len(lists) == 2:
        (word, label) = lists
        relation = ''
    elif len(lists) == 3:
        (word, label, relation) = lists
    else:
        print('illegal data line:{}'.format(i + 1))
        continue

    if label == 'O':
        outs.append(word + item_sep + 'O' + item_sep + relation)
    elif label.startswith('S-'):
        outs.append(word + item_sep + label[2:] + item_sep + relation)
    elif label.startswith('B-'):
        pre_word = word
    elif label.startswith('I-'):
        pre_word = pre_word + word_sep + word
    elif label.startswith('E-'):
        pre_word = pre_word + word_sep + word
        outs.append(pre_word + item_sep + label[2:] + item_sep + relation)
    else:
        pass
    
pprint.pprint(outs)

['AAA O ',
 'BBBBB O ',
 'CC_DDD_EEE_FFFFF GOODSNAME 1_2',
 'G O ',
 'H O ',
 'I O ',
 'J O ',
 'KKK O ',
 'LL O ',
 'MMMM ORDERCURRENCY 1',
 'NNNNN ORDERPRICE 1',
 'OOOO_PPPPPP ORDERUNIT 1',
 'QQQQQQQQQQQQ O ',
 'RR O ',
 'SS O ',
 'TT O ',
 'UU O ',
 'V ORDERCURRENCY 2',
 'W ORDERUNIT 2',
 'X O ',
 'YYY_ZZZZZZZZZZZZZZZZ ORDERUNIT 3']


In [4]:
str_out = ''
word_sep = ' '

for i, data in enumerate(outs):
    
    (word, label, relation) = re.sub(r'\n$', '', data).split(' ')
    
    if label != 'O':
        tag1 = r'<' + label + r'>'
        tag2 = r'</' + label + r'>'
        str_out = str_out + word_sep + tag1 + word + tag2
    else:
        str_out = str_out + word_sep + word

print(str_out)

 AAA BBBBB <GOODSNAME>CC_DDD_EEE_FFFFF</GOODSNAME> G H I J KKK LL <ORDERCURRENCY>MMMM</ORDERCURRENCY> <ORDERPRICE>NNNNN</ORDERPRICE> <ORDERUNIT>OOOO_PPPPPP</ORDERUNIT> QQQQQQQQQQQQ RR SS TT UU <ORDERCURRENCY>V</ORDERCURRENCY> <ORDERUNIT>W</ORDERUNIT> X <ORDERUNIT>YYY_ZZZZZZZZZZZZZZZZ</ORDERUNIT>


### 以下、O-tag用処理

In [51]:
O_TAG_LABEL = 'RandomOtag'

def make_Otag(buf, idx_low, idx_up):

    list2 = []
    for i, data in enumerate(buf):
        lists = re.sub(r'\n$', '', data).split(' ')
        if len(lists) == 2:
            (word, label) = lists
            #print(str(i) + '\t' + word + '\t' + label)
        else:
            (word, label, tmp) = lists
            #print(str(i) + '\t' + word + '\t' + label)

        if label == 'O' and i in list_otag:
            if i == idx_low and i ==  idx_up:
                list2.append(word + ' ' + 'S-' + O_TAG_LABEL)
            elif i == idx_low:
                list2.append(word + ' ' + 'B-' + O_TAG_LABEL)
            elif i == idx_up:
                list2.append(word + ' ' + 'E-' + O_TAG_LABEL)
            else:
                list2.append(word + ' ' + 'I-' + O_TAG_LABEL)
        else:
            list2.append(word + ' ' + label)

    #pprint.pprint(list2)

    outs = []
    pre_word = ''
    pre_label = ''

    word_sep = '_'
    item_sep = ' ' 

    for i, data in enumerate(list2):

        lists = re.sub(r'\n$', '', data).split(' ')

        if len(lists) == 2:
            (word, label) = lists
            relation = ''
        elif len(lists) == 3:
            (word, label, relation) = lists
        else:
            print('illegal data line:{}'.format(i + 1))
            continue

        if label == 'O':
            outs.append(word + item_sep + 'O' + item_sep + relation)
        elif label.startswith('S-'):
            outs.append(word + item_sep + label[2:] + item_sep + relation)
        elif label.startswith('B-'):
            pre_word = word
        elif label.startswith('I-'):
            pre_word = pre_word + word_sep + word
        elif label.startswith('E-'):
            pre_word = pre_word + word_sep + word
            outs.append(pre_word + item_sep + label[2:] + item_sep + relation)
        else:
            pass

    #pprint.pprint(outs)

    str_out = ''
    word_sep = ' '

    for i, data in enumerate(outs):

        (word, label, relation) = re.sub(r'\n$', '', data).split(' ')

        if label != 'O':
            tag1 = r'<' + label + r'>'
            tag2 = r'</' + label + r'>'
            str_out = str_out + word_sep + tag1 + word + tag2
        else:
            str_out = str_out + word_sep + word

    #print(str_out)
    
    return str_out

In [52]:
# 付与するO-tagをランダムに取得
list_otag = [7, 8, 9, 10]
#list_otag = [7, 8]
#list_otag = [9]
idx_low = list_otag[0]
idx_up = list_otag[len(list_otag) - 1]
print('idx_low:{}'.format(idx_low))
print('idx_up:{}'.format(idx_up))

idx_low:7
idx_up:10


In [53]:
ret = make_Otag(buf, idx_low, idx_up)
print(ret)

 AAA BBBBB <GOODSNAME>CC_DDD_EEE_FFFFF</GOODSNAME> G <RandomOtag>H_I_J_KKK</RandomOtag> LL <ORDERCURRENCY>MMMM</ORDERCURRENCY> <ORDERPRICE>NNNNN</ORDERPRICE> <ORDERUNIT>OOOO_PPPPPP</ORDERUNIT> QQQQQQQQQQQQ RR SS TT UU <ORDERCURRENCY>V</ORDERCURRENCY> <ORDERUNIT>W</ORDERUNIT> X <ORDERUNIT>YYY_ZZZZZZZZZZZZZZZZ</ORDERUNIT>
